This is the data ingestion & exploration notebook. Please take note that the Explore_Amazon_Data module will read in and modify existing tables if you give it an existing table name.

In [ ]:
import pandas as pd
import os 
import gzip
import sqlite3 as sq
import Explore_Amazon_Data as amzn  #This is our data ingestion module, please ensure that this python file is in the same folder as the Notebook.

The two cells below do two things: 
1.) Defines the three inputs into the Explore_Amazon_Data module. amazon_file_path is the location of the raw aws customer review data. db_file_path is the location of the SQLite DB. table_name defines which table the data will be dumped into.

2.) Run the ingestion module to read in raw files and dump to the specified tables in the SQLite database.

In [ ]:
amazon_file_path = #file path of aws customer review data goes here
db_file_path = #file path of SQLite database
table_name = #table name to store data in SQLite database

In [ ]:
#amzn.create(amazon_file_path, db_file_path, table_name)

The below cell is the beginning of the data exploration. It sets up a connection and a cursor. You can then write SQL queries (as shown below) to pull data from the DB.

In [ ]:
conn = sq.connect() #sqliteDB path goes in parantheses
crsr = conn.cursor()

crsr.execute('''
                SELECT 
                    marketplace, 
                    customer_id 
                from amazon_music_reviews LIMIT 5''')

print(crsr.fetchall())